In [2]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate

In [3]:
load_dotenv()
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [4]:
def get_pdf_text(pdf_docs):
    loader = PyPDFLoader(file_path=pdf_docs)
    pages = loader.load()
    return pages

In [5]:
def get_text_chunks(pages):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1500, chunk_overlap = 200, 
        length_function=len, separators=['\n\n', '\n', ' ', '', '.', ',']
    )
    chunks = text_splitter.split_documents(pages)
    
    return chunks

In [6]:
def get_all_text(chunks_data):
    page_contents = []
    for chunk in chunks_data:
        page_contents.append(chunk.page_content)
    return ' '.join(page_contents)

In [7]:
def get_pdf_chain(pdf_text):
    
    prompt_pdf = PromptTemplate.from_template(
    """
    ### Resume Information Extraction

    Extract the following information from the provided resume text:

    - Name
    - Email
    - Skills
    - Projects

    Return the extracted information in a JSON format without any preamble.

    Extracted Text: {pdf_text}
    """
    )
    
    llm = ChatGroq(
        temperature=0,
        model="llama-3.1-70b-versatile",
        api_key=GROQ_API_KEY
    )
    
    chain_pdf = prompt_pdf | llm
    result = chain_pdf.invoke(input={'pdf_text': pdf_text})
    
    return result.content

In [8]:
pages = get_pdf_text(pdf_docs="Praveen_S_CV.pdf")
chunks = get_text_chunks(pages=pages)
pdf_text = get_all_text(chunks_data=chunks)
chain_pdf = get_pdf_chain(pdf_text=pdf_text)

In [11]:
def get_web_chain(web_text):
    
    prompt_web = PromptTemplate.from_template(
    """
    ### Job Posting Extraction

    Extract the following information from the provided job posting text:

    - Role
    - Experience
    - Skills
    - Description

    Return the extracted information in a JSON format without any preamble.

    Job Posting Text: {web_text}
    """
    )
    
    llm = ChatGroq(
        temperature=0,
        model="llama-3.1-70b-versatile",
        api_key=GROQ_API_KEY
    )
    
    chain_web = prompt_web | llm
    result = chain_web.invoke(input={'web_text': web_text})
    
    return result.content

In [24]:
def get_web_text(url):
    loader = WebBaseLoader(web_path=url)
    pages_data = loader.load().pop().page_content
    return pages_data

In [59]:
web_text = get_web_text(url="https://sarvm.freshteam.com/jobs/1uuNcm-EuTUd/ai-ml-intern-fresher-remote")
chain_web = get_web_chain(web_text=web_text)

In [61]:
def get_email_chain(chain_web, chain_pdf):
    
    prompt_email = PromptTemplate.from_template(
    """
    ### Cold Email Generation

    Generate a cold email to the hiring manager based on the provided job posting and resume information.

    Use the following information to generate the email:

    Job Posting: {job_description}
    Resume Information: {resume_data}

    Write a persuasive email that highlights the candidate's skills and experience, and why they are the best fit for the job.

    Email should not include any preamble or the candidate's email address.

    Returns a well-structured email in plain text format.
    """
    )
    
    llm = ChatGroq(
        temperature=0,
        model="llama-3.1-70b-versatile",
        api_key=GROQ_API_KEY
    )
    
    chain_email = prompt_email | llm
    result = chain_email.invoke(input={'job_description': chain_web, 
                                       'resume_data': chain_pdf})
    
    return result.content

In [62]:
generator = get_email_chain(chain_web=chain_web, chain_pdf=chain_pdf)

In [63]:
print(generator)

Subject: Application for AI-ML Intern/Fresher Role

Dear Hiring Manager,

I am excited to apply for the AI-ML Intern/Fresher position at your organization, as advertised. With a strong passion for artificial intelligence and machine learning, I am confident that my skills and experience make me an ideal candidate for this role.

As a proficient Python programmer with expertise in deep learning frameworks such as TensorFlow and Keras, I am well-equipped to handle the responsibilities of this position. My experience with scikit-learn, pandas, and OpenCV has allowed me to develop a strong foundation in data preprocessing, feature engineering, and model development. Additionally, my familiarity with Linux and hardware selection for ML model latency will enable me to optimize model performance and deployment.

In my previous projects, I have demonstrated my ability to lead end-to-end machine learning processes, from data collection and preprocessing to model training and deployment. For ins